##  Predicting Baseball Hall of Fame Induction

###  Data Analysis

####  Eryk Wdowiak and Ken Hoffman

data from Lahman Baseball Database

In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import pickle

import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

# from sklearn.linear_model import LogisticRegression 
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import VotingClassifier ##, BaggingClassifier, RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, accuracy_score

# from imblearn.over_sampling import SMOTE

# import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
##  the little date string
dt_str = '2020-08-19a'

# get the dataframes from pickle files
pitchers_df = pickle.load(open('pitchers-df_'+ dt_str +'.p','rb'))
catchers_df = pickle.load(open('catchers-df_'+ dt_str +'.p','rb'))
infielders_df = pickle.load(open('infielders-df_'+ dt_str +'.p','rb'))
outfielders_df = pickle.load(open('outfielders-df_'+ dt_str +'.p','rb'))

In [3]:
def bball_log(x):
    warnings.filterwarnings('ignore',category=RuntimeWarning)
    return np.where(x<1,0,np.log(x))

In [4]:
def dummies(x, prefix):
    return pd.get_dummies(x, prefix = prefix, drop_first = False)

### pitchers

In [5]:
##  take logs
pitchers_df['ln_G'] = bball_log(pitchers_df['G'])
pitchers_df['ln_G_ps'] = bball_log(pitchers_df['G_ps'])
pitchers_df['ln_W'] = bball_log(pitchers_df['W'])
pitchers_df['ln_W_ps'] = bball_log(pitchers_df['W_ps'])
pitchers_df['ln_SO'] = bball_log(pitchers_df['SO'])
pitchers_df['ln_SO_ps'] = bball_log(pitchers_df['SO_ps'])
pitchers_df['ln_ER'] = bball_log(pitchers_df['ER'])
pitchers_df['ln_ER_ps'] = bball_log(pitchers_df['ER_ps'])
pitchers_df['ln_IPouts'] = bball_log(pitchers_df['IPouts'])
pitchers_df['ln_IPouts_ps'] = bball_log(pitchers_df['IPouts_ps'])

##  not sure about this one
pitchers_df['ln_nu_sns'] = bball_log(pitchers_df['nu_sns'])

##  years since retirement
pitchers_df['since_lst'] = 2018 - pitchers_df['lst_sn']
pitchers_df['ln_since'] = np.log(pitchers_df['since_lst'])

##  create dummies for teams
team_dums = dummies(pitchers_df['teamID'],'tm')
pitchers_df = pitchers_df.join(team_dums)

##  just need the list
team_cols = team_dums.columns

In [6]:
##  exclude teams without inductees
tm_excl = ['tm_ANA', 'tm_ARI', 'tm_BRO', 'tm_BSN', 'tm_CIN', 'tm_COL', 
           'tm_FLO', 'tm_HOU', 'tm_KC1', 'tm_KCA', 'tm_LAA', 'tm_MIL', 
           'tm_ML4', 'tm_MON', 'tm_NY1', 'tm_PHA', 'tm_PIT', 'tm_SLA', 
           'tm_TBA', 'tm_TEX', 'tm_TOR', 'tm_WAS', 'tm_WS1', 'tm_WS2']

##  make everything relative to the Yankees
tm_excl = tm_excl + ['tm_NYA']

##  columns to include
tm_incl = [dum for dum in team_cols if dum not in tm_excl]

##  list of exogenous variables for regression model
exog = ['ln_G']
exog = exog + ['ln_G_ps']
exog = exog + ['ln_W']
exog = exog + ['ln_W_ps']
exog = exog + ['ln_SO']
exog = exog + ['ln_SO_ps']
exog = exog + ['ln_ER']
exog = exog + ['ln_ER_ps']
exog = exog + ['ln_IPouts']
exog = exog + ['ln_IPouts_ps']
exog = exog + ['ln_nu_sns']
exog = exog + ['ln_since']
exog = exog + tm_incl

#list(team_cols[[dum not in tm_excl for dum in team_cols]])

##  regression formula
m01a_fmla = 'induct~'
m01a_fmla = m01a_fmla + '+'.join(exog)

##  run logit
m01a_lgt = smf.logit(m01a_fmla,data=pitchers_df).fit() 
m01a_lgt.summary()

Optimization terminated successfully.
         Current function value: 0.024227
         Iterations 16


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 induct   No. Observations:                  900
Model:                          Logit   Df Residuals:                      869
Method:                           MLE   Df Model:                           30
Date:                Wed, 19 Aug 2020   Pseudo R-squ.:                  0.8587
Time:                        12:58:14   Log-Likelihood:                -21.804
converged:                       True   LL-Null:                       -154.31
                                        LLR p-value:                 1.866e-39
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -29.1007     30.663     -0.949      0.343     -89.199      30.998
ln_G             7.3710      4.546      1.621      0.105      -1.539      16.281
ln_G_ps         -2.7617      1.867     -1.480      0.139      -6.420       0.897
ln_W            53.3876     20.081      2.659      0.008      14.030      92.745
ln_W_ps          1.4232      1.254      1.135      0.256      -1.035       3.881
ln_SO            3.0498      3.484      0.875      0.381      -3.778       9.877
ln_SO_ps         4.8338      2.843      1.700      0.089      -0.738      10.406
ln_ER          -32.6699     10.680     -3.059      0.002     -53.603     -11.737
ln_ER_ps        -2.1209      1.668     -1.271      0.204      -5.390       1.149
ln_IPouts      -13.3668     11.783     -1.134      0.257     -36.462       9.728
ln_IPouts_ps    -1.8364      1.439     -1.276      0.202      -4.656       0.983
ln_nu_sns        4.8419      7.093      0.683      0.495      -9.060      18.743
ln_since         3.7207      1.924      1.934      0.053      -0.051       7.492
tm_ATL           9.4898      5.445      1.743      0.081      -1.181      20.161
tm_BAL          -3.5476      4.353     -0.815      0.415     -12.080       4.984
tm_BOS          -7.3174      4.144     -1.766      0.077     -15.440       0.805
tm_CAL           3.4972     78.087      0.045      0.964    -149.551     156.545
tm_CHA           1.2845      2.168      0.592      0.554      -2.965       5.534
tm_CHN          11.2222      4.170      2.691      0.007       3.050      19.395
tm_CLE           1.2605      2.396      0.526      0.599      -3.435       5.956
tm_DET           3.4306      2.274      1.508      0.131      -1.027       7.888
tm_LAN           3.2334      3.385      0.955      0.339      -3.401       9.868
tm_MIN           2.4459      2.659      0.920      0.358      -2.765       7.657
tm_ML1          -4.4152     20.891     -0.211      0.833     -45.362      36.531
tm_NYN          -3.5630     12.216     -0.292      0.771     -27.507      20.381
tm_OAK           9.7072      3.841      2.527      0.011       2.179      17.235
tm_PHI           1.9096      2.673      0.714      0.475      -3.329       7.149
tm_SDN          20.2112      6.665      3.033      0.002       7.149      33.274
tm_SEA           2.8013     70.589      0.040      0.968    -135.550     141.152
tm_SFN          -1.0111      3.087     -0.328      0.743      -7.062       5.040
tm_SLN          -0.0403      2.063     -0.020      0.984      -4.083       4.002
================================================================================

Possibly complete quasi-separation: A fraction 0.86 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""